In [3]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from mpi4py import MPI

def f(x):
    return 1/(1 + x**2)

def trapz(f, a, b, n_points = 1):
    
    dx = (b - a)/n_points
    ans = 1/2 * (f(a) + f(b))
    
    for i in range(1, int(n_points)):
        ans += f(a + i*dx)     
    return ans*dx

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = -pi/4
b = pi/4
disc = 1e+5
my_int = np.zeros(1)
integral_sum = np.zeros(1)

start = MPI.Wtime()

# Compute partition
dx = (b - a) / disc # calculate dx 

if disc >= size:
    if rank != size - 1:
        discret_number = int(np.floor(disc/size))
    else:
        discret_number = int(disc - np.floor(disc*(size - 1)/size))
else:
    if rank < disc:
        discret_number = 1
    else:
        discret_number = 0

#print('I m process', rank, 'number of points', discret_number)
dx = (b - a)/disc
a_i = a + rank * dx * discret_number
b_i = a + (rank + 1) * dx * discret_number
#print("The first and second points:", a_i, b_i)
my_int[0] = trapz(f, a_i, b_i, discret_number)

comm.Barrier()
end = MPI.Wtime()

# Send partition back to root process, computing sum across all partitions
#print("Process ", rank, " has the partial integral ", my_int[0])
comm.Reduce(my_int, integral_sum, MPI.SUM, root = 0)

# Only print the result in process 0
if rank == 0:
    #an_int = analytical_integral(a, b)
    #accuracy = (an_int - integral_sum[0])/an_int
    #print('The Integral Sum =', integral_sum[0])
    #print('Analytical Integral Sum = ', an_int, '\nAccuracy =', abs(accuracy))
    print(size, end - start)

1 0.023873596999919755
